# forecast, distribution plots

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import warnings
from scipy import stats
warnings.filterwarnings('ignore')

class StatsBasedVisualization:
    """
    Create clean forecast visualizations using compressed statistics files
    Adapted for local execution
    """
    
    def __init__(self):
        # Local paths - working in current directory
        self.base_path = Path('.')  # Current directory
        self.portfolio_path = self.base_path / 'Renewable Portfolio LLC'
        
        # Check if path exists
        if self.portfolio_path.exists():
            print(f"✅ Found portfolio at: {self.portfolio_path}")
        else:
            print(f"⚠️  Portfolio path not found at: {self.portfolio_path}")
            print("   Please verify the path structure in your current directory")
        
        # Modern color palette
        self.colors = {
            'generation': '#70AD47',  # Green
            'price': '#4472C4',       # Blue for real-time
            'price_da': '#ED7D31',    # Orange for day-ahead
            'revenue': '#ED7D31',     # Orange
        }
        
        # Month names
        self.month_names = ['January', 'February', 'March', 'April', 'May', 'June',
                           'July', 'August', 'September', 'October', 'November', 'December']
    
    def get_project_folders(self):
        """Get all project folders (excluding notebooks and other files)"""
        project_folders = []
        if not self.portfolio_path.exists():
            return project_folders
            
        for item in self.portfolio_path.iterdir():
            if item.is_dir():
                # Check if it has the expected subfolders (including Price_da)
                if ((item / 'Generation').exists() or (item / 'Price').exists() or 
                    (item / 'Price_da').exists() or (item / 'Revenue').exists()):
                    project_folders.append(item)
        return project_folders
    
    def clean_site_name(self, site_name):
        """Clean up site name for display"""
        # Remove LLC suffixes and underscores for cleaner display
        clean_name = site_name.replace('_LLC', '').replace('_Power', '')
        clean_name = clean_name.replace('_', ' ').title()
        return clean_name
    
    def get_all_sites(self):
        """Get all unique site names from the project folders"""
        return [folder.name for folder in self.get_project_folders()]
    
    def format_y_axis(self, ax, metric_type, temporal):
        """Format y-axis based on metric type and temporal resolution"""
        if metric_type == 'generation':
            if temporal in ['daily', 'monthly']:
                ax.set_ylabel('Generation (MWh)', fontsize=11)
            else:
                ax.set_ylabel('Generation (MW)', fontsize=11)
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
        
        elif metric_type == 'price' or metric_type == 'price_da':
            ax.set_ylabel('Price ($/MWh)', fontsize=11)
            ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
        
        elif metric_type == 'revenue':
            if temporal == 'monthly':
                ax.set_ylabel('Revenue ($1000s)', fontsize=11)
                ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1000:,.0f}'))
            else:
                ax.set_ylabel('Revenue ($)', fontsize=11)
                ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
    
    def find_stats_file(self, project_folder, metric_type, temporal):
        """Find the appropriate stats file in the project folder"""
        metric_folder = project_folder / metric_type.capitalize()
        
        if not metric_folder.exists():
            return None
        
        # Look for stats files with various naming patterns
        patterns = [
            f'*_{metric_type}_{temporal}_stats.csv',
            f'*_{temporal}_stats.csv',
            f'*{temporal}stats.csv'
        ]
        
        for pattern in patterns:
            files = list(metric_folder.glob(pattern))
            if files:
                return files[0]
        
        return None
    
    def find_timeseries_file(self, project_folder, metric_type, temporal):
        """Find the appropriate timeseries file in the project folder"""
        metric_folder = project_folder / metric_type.capitalize()
        
        if not metric_folder.exists():
            return None
        
        # Look for timeseries files with various naming patterns
        patterns = [
            f'*_{metric_type}_{temporal}_timeseries.csv',
            f'*_{temporal}_timeseries.csv',
            f'*{temporal}timeseries.csv',
            f'*_{metric_type}_{temporal}_timeseries_compressed.csv'
        ]
        
        for pattern in patterns:
            files = list(metric_folder.glob(pattern))
            if files:
                return files[0]
        
        return None
    
    def plot_monthly_duration_curves(self, site_name, metric_type):
        """Create monthly duration curves - ONLY FOR PRICE"""
        if metric_type != 'price':
            return False
        
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
            
        # Construct the fixed filename for compressed hourly timeseries
        price_folder = project_folder / 'Price'
        if not price_folder.exists():
            return False
            
        # Fixed filename format: {site_name}_price_hourly_timeseries_compressed.csv
        timeseries_file = price_folder / f"{site_name}_price_hourly_timeseries_compressed.csv"
        
        if not timeseries_file.exists():
            print(f"      Warning: Compressed file not found: {timeseries_file.name}")
            return False
        
        try:
            # Read the hourly data
            df = pd.read_csv(timeseries_file)
            
            # Create duration_curves subfolder inside site's plots folder
            plots_folder = project_folder / 'plots'
            duration_folder = plots_folder / 'duration_curves'
            duration_folder.mkdir(parents=True, exist_ok=True)
            
            # Extract all year columns (numeric columns)
            year_cols = [col for col in df.columns if str(col).isdigit()]
            
            if not year_cols:
                return False
            
            # Process each month separately
            created_count = 0
            
            for month_idx in range(1, 13):
                # Filter data for this month
                month_data = df[df['month'] == month_idx].copy()
                
                if len(month_data) == 0:
                    continue
                
                # Collect all hourly values for this month across all years
                month_values = []
                for year in year_cols:
                    year_values = pd.to_numeric(month_data[year], errors='coerce').dropna()
                    month_values.extend(year_values.tolist())
                
                if not month_values:
                    continue
                
                # Sort values in descending order for duration curve
                sorted_values = np.sort(month_values)[::-1]
                
                # Calculate duration percentages
                n_values = len(sorted_values)
                duration_pct = np.linspace(0, 100, n_values)
                
                # Calculate key percentiles
                percentiles_to_mark = [1, 5, 25, 50, 75, 95, 99]
                percentile_values = {}
                percentile_indices = {}
                
                for p in percentiles_to_mark:
                    idx = int((100 - p) / 100 * n_values)
                    idx = min(idx, n_values - 1)
                    percentile_values[p] = sorted_values[idx]
                    percentile_indices[p] = idx
                
                # Calculate mean
                mean_value = np.mean(month_values)
                
                # Create the plot
                fig, ax = plt.subplots(figsize=(10, 6))
                ax.set_facecolor('white')
                fig.patch.set_facecolor('white')
                
                # Plot the duration curve
                ax.plot(duration_pct, sorted_values, 
                       color='red', linewidth=2, zorder=5)
                
                # Add mean line
                ax.axhline(y=mean_value, color='black', linestyle='--', 
                          linewidth=1.2, alpha=0.8)
                
                # Position mean label
                y_range = max(sorted_values) - min(sorted_values)
                mean_label_y = mean_value + y_range * 0.02
                
                if mean_label_y > max(sorted_values) * 0.95:
                    mean_label_y = mean_value - y_range * 0.02
                
                mean_label = f'Mean: ${mean_value:.2f}'
                
                ax.text(50, mean_label_y, mean_label, 
                       ha='center', va='bottom' if mean_label_y > mean_value else 'top', 
                       fontsize=10, fontweight='bold')
                
                # Mark percentiles
                percentile_colors = {
                    99: 'green', 95: 'green', 75: 'green',
                    50: 'blue', 25: 'orange', 5: 'red', 1: 'red'
                }
                
                for p in percentiles_to_mark:
                    idx = percentile_indices[p]
                    value = percentile_values[p]
                    duration = duration_pct[idx]
                    
                    # Add marker
                    ax.plot(duration, value, 'o', 
                           color=percentile_colors.get(p, 'gray'),
                           markersize=6, zorder=10)
                    
                    # Add label
                    label = f'P{p}'
                    value_text = f'${value:.1f}'
                    
                    # Position text
                    y_range = max(sorted_values) - min(sorted_values)
                    if p >= 50:
                        va = 'bottom'
                        y_offset = value + y_range * 0.015
                    else:
                        va = 'top'
                        y_offset = value - y_range * 0.015
                    
                    # Add percentile label
                    ax.text(duration, y_offset, label, 
                           ha='center', va=va, fontsize=8, 
                           color=percentile_colors.get(p, 'gray'),
                           fontweight='bold')
                    
                    # Add value box
                    bbox_props = dict(boxstyle="round,pad=0.2", 
                                    facecolor='white', 
                                    edgecolor='black',
                                    linewidth=0.8)
                    ax.text(duration, value, value_text,
                           ha='center', va='center', fontsize=8,
                           bbox=bbox_props, zorder=11)
                
                # Title and labels
                clean_name = self.clean_site_name(site_name)
                month_name = self.month_names[month_idx - 1]
                
                title = f'Price Duration Curve - {month_name} - {clean_name}'
                if min(sorted_values) < 0:
                    title += '\n(includes negative prices)'
                
                ax.set_title(title, fontsize=12, fontweight='normal', pad=10)
                ax.set_xlabel('Duration (% of time)', fontsize=11)
                ax.set_ylabel('Price ($ per MWh)', fontsize=11)
                
                # Set x-axis
                ax.set_xlim(0, 100)
                ax.set_xticks(range(0, 101, 20))
                
                # Y-axis formatting
                y_min = min(sorted_values) * 1.1 if min(sorted_values) < 0 else 0
                y_max = max(sorted_values) * 1.1
                ax.set_ylim(y_min, y_max)
                ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
                
                # Add zero line if there are negative prices
                if min(sorted_values) < 0:
                    ax.axhline(y=0, color='gray', linestyle='-', linewidth=1, alpha=0.5, zorder=3)
                
                # Grid
                ax.grid(True, axis='both', alpha=0.3, linestyle='-', linewidth=0.5)
                
                # Spines
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_color('lightgray')
                ax.spines['bottom'].set_color('lightgray')
                
                plt.tight_layout()
                
                # Save
                month_name_lower = month_name.lower()
                output_path = duration_folder / f'{metric_type}_duration_{month_name_lower}.png'
                plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
                plt.close()
                
                created_count += 1
            
            return created_count > 0
            
        except Exception as e:
            print(f"Error in monthly duration curves for {site_name} {metric_type}: {str(e)}")
            return False
    
    def plot_monthly_distribution_curves(self, site_name, metric_type):
        """Create monthly distribution curves - FOR GENERATION AND REVENUE"""
        if metric_type not in ['generation', 'revenue']:
            return False
        
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
            
        # Find monthly timeseries file
        timeseries_file = self.find_timeseries_file(project_folder, metric_type, 'monthly')
        
        if not timeseries_file:
            return False
        
        try:
            # Read the monthly timeseries data
            df = pd.read_csv(timeseries_file)
            
            # Create distribution_curves subfolder inside site's plots folder
            plots_folder = project_folder / 'plots'
            dist_folder = plots_folder / 'distribution_curves'
            dist_folder.mkdir(parents=True, exist_ok=True)
            
            # Extract all year columns
            year_cols = [col for col in df.columns if str(col).isdigit()]
            
            if not year_cols:
                return False
            
            # Process each month separately
            created_count = 0
            
            for month_idx in range(1, 13):
                # Filter data for this month
                month_data = df[df['month'] == month_idx].copy()
                
                if len(month_data) == 0:
                    continue
                
                # Collect all monthly values
                month_values = []
                for year in year_cols:
                    year_value = pd.to_numeric(month_data[year].iloc[0], errors='coerce')
                    if not pd.isna(year_value):
                        month_values.append(year_value)
                
                if len(month_values) < 3:
                    continue
                
                # Convert to numpy array
                month_values = np.array(month_values)
                
                # Calculate statistics
                mean_val = np.mean(month_values)
                median_val = np.median(month_values)
                std_val = np.std(month_values)
                p5_val = np.percentile(month_values, 5)
                p95_val = np.percentile(month_values, 95)
                
                # Create the plot
                fig, ax = plt.subplots(figsize=(10, 7))
                ax.set_facecolor('white')
                fig.patch.set_facecolor('white')
                
                # Create x range for plotting
                x_min = min(month_values) - 0.2 * (max(month_values) - min(month_values))
                x_max = max(month_values) + 0.2 * (max(month_values) - min(month_values))
                x_smooth = np.linspace(x_min, x_max, 300)
                
                # Calculate KDE
                from scipy.stats import gaussian_kde
                kde = gaussian_kde(month_values, bw_method='scott')
                kde_values = kde(x_smooth)
                
                # Plot the KDE curve
                ax.plot(x_smooth, kde_values, 
                       color=self.colors[metric_type],
                       linewidth=3,
                       label='KDE Distribution',
                       zorder=5)
                
                # Fill under the KDE curve
                ax.fill_between(x_smooth, 0, kde_values,
                              alpha=0.3,
                              color=self.colors[metric_type],
                              edgecolor='none')
                
                # Plot normal distribution for comparison
                normal_dist = stats.norm.pdf(x_smooth, mean_val, std_val)
                ax.plot(x_smooth, normal_dist, 'k--', linewidth=2, 
                       label=f'Normal fit (μ={mean_val:,.0f}, σ={std_val:,.0f})')
                
                # Add scatter points for actual data
                y_jitter = np.random.uniform(0, max(kde_values) * 0.05, size=len(month_values))
                ax.scatter(month_values, y_jitter, 
                          color=self.colors[metric_type],
                          alpha=0.6,
                          s=50,
                          edgecolors='black',
                          linewidth=1,
                          label=f'Actual values (n={len(month_values)})',
                          zorder=10)
                
                # Add vertical lines for key statistics
                ax.axvline(x=mean_val, color='red', linestyle='--', linewidth=2, 
                          label=f'Mean: {mean_val:,.0f}')
                ax.axvline(x=median_val, color='blue', linestyle='--', linewidth=2, 
                          label=f'Median: {median_val:,.0f}')
                ax.axvline(x=p5_val, color='green', linestyle=':', linewidth=1.5, 
                          label=f'P5: {p5_val:,.0f}')
                ax.axvline(x=p95_val, color='green', linestyle=':', linewidth=1.5, 
                          label=f'P95: {p95_val:,.0f}')
                
                # Title and labels
                clean_name = self.clean_site_name(site_name)
                month_name = self.month_names[month_idx - 1]
                
                if metric_type == 'generation':
                    title = f'Monthly Generation Distribution - {month_name} - {clean_name}'
                    ax.set_xlabel('Monthly Generation (MWh)', fontsize=11)
                    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:,.0f}'))
                else:  # revenue
                    title = f'Monthly Revenue Distribution - {month_name} - {clean_name}'
                    ax.set_xlabel('Monthly Revenue ($)', fontsize=11)
                    ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))
                
                title += f'\n(Distribution across {len(month_values)} simulation years)'
                ax.set_title(title, fontsize=12, fontweight='normal', pad=10)
                ax.set_ylabel('Probability Density', fontsize=11)
                
                # Set y-axis to start at 0
                ax.set_ylim(bottom=0)
                
                # Grid
                ax.grid(True, axis='both', alpha=0.3, linestyle='-', linewidth=0.5)
                
                # Spines
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_color('lightgray')
                ax.spines['bottom'].set_color('lightgray')
                
                # Legend
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                         ncol=3, frameon=False, fontsize=9)
                
                # Add text box with statistics
                cv = (std_val / mean_val) * 100 if mean_val != 0 else 0
                textstr = f'Years: {len(month_values)}\nStd Dev: {std_val:,.0f}\nCV: {cv:.1f}%\nMin: {min(month_values):,.0f}\nMax: {max(month_values):,.0f}'
                
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
                ax.text(0.02, 0.95, textstr, transform=ax.transAxes, fontsize=9,
                       verticalalignment='top', bbox=props)
                
                plt.tight_layout()
                plt.subplots_adjust(bottom=0.15)
                
                # Save
                month_name_lower = month_name.lower()
                output_path = dist_folder / f'{metric_type}_distribution_{month_name_lower}.png'
                plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
                plt.close()
                
                created_count += 1
            
            return created_count > 0
            
        except Exception as e:
            print(f"Error in monthly distribution curves for {site_name} {metric_type}: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def plot_monthly_forecast(self, site_name, metric_type):
        """Create monthly forecast plot using timeseries data"""
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
            
        # Find monthly timeseries file
        timeseries_file = self.find_timeseries_file(project_folder, metric_type, 'monthly')
        
        if not timeseries_file:
            # Try to find stats file as fallback
            stats_file = self.find_stats_file(project_folder, metric_type, 'monthly')
            if stats_file:
                return self.plot_monthly_forecast_from_stats(site_name, metric_type, stats_file)
            return False
        
        try:
            # Read the timeseries data
            df = pd.read_csv(timeseries_file)
            
            # Get year columns
            year_cols = [col for col in df.columns if str(col).isdigit()]
            
            if not year_cols:
                return False
            
            # Calculate statistics
            df['mean'] = df[year_cols].mean(axis=1)
            df['p5'] = df[year_cols].quantile(0.05, axis=1)
            df['p95'] = df[year_cols].quantile(0.95, axis=1)
            
            # Create the plot
            fig, ax = plt.subplots(figsize=(12, 7))
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')
            
            # X positions
            x = range(len(df))
            
            # Check if this is price
            if metric_type == 'price':
                # Plot real-time price mean
                ax.plot(x, df['mean'], 
                       color=self.colors['price'],
                       linewidth=3,
                       label='Real-Time Price',
                       zorder=5,
                       marker='o',
                       markersize=6)
                
                # Try to load day-ahead price data from Price_da folder
                price_da_folder = project_folder / 'Price_da'
                if price_da_folder.exists():
                    # Look for monthly timeseries in Price_da
                    dt_patterns = [
                        '*_monthly_timeseries.csv',
                        '*monthly_timeseries.csv',
                        '*_price_da_monthly_timeseries.csv'
                    ]
                    
                    dt_file = None
                    for pattern in dt_patterns:
                        files = list(price_da_folder.glob(pattern))
                        if files:
                            dt_file = files[0]
                            break
                    
                    if dt_file:
                        try:
                            df_dt = pd.read_csv(dt_file)
                            year_cols_dt = [col for col in df_dt.columns if str(col).isdigit()]
                            
                            if year_cols_dt:
                                df_dt['mean'] = df_dt[year_cols_dt].mean(axis=1)
                                
                                # Plot day-ahead price
                                ax.plot(x, df_dt['mean'], 
                                       color=self.colors['price_da'],
                                       linewidth=3,
                                       label='Day-Ahead Price',
                                       zorder=5,
                                       marker='s',
                                       markersize=6,
                                       linestyle='--')
                        except Exception as e:
                            print(f"      Note: Could not load day-ahead price data: {e}")
                
            else:
                # For generation and revenue, plot with confidence band
                ax.fill_between(x, df['p5'], df['p95'],
                              alpha=0.3,
                              color=self.colors[metric_type],
                              label='P5-P95 Confidence Band',
                              edgecolor='none')
                
                ax.plot(x, df['mean'], 
                       color=self.colors[metric_type],
                       linewidth=3,
                       label='Mean',
                       zorder=5,
                       marker='o',
                       markersize=6)
            
            # Title
            clean_name = self.clean_site_name(site_name)
            title = f'Monthly {metric_type.capitalize()} Forecast - {clean_name}'
            ax.set_title(title, fontsize=14, fontweight='normal', pad=15)
            
            # X-axis
            ax.set_xticks(x)
            if 'month_name' in df.columns:
                ax.set_xticklabels(df['month_name'], rotation=45, ha='right')
            else:
                ax.set_xticklabels([self.month_names[i] for i in range(12)], rotation=45, ha='right')
            ax.set_xlabel('')
            
            # Y-axis
            self.format_y_axis(ax, metric_type, 'monthly')
            
            # Grid
            ax.grid(True, axis='y', alpha=0.3, linestyle='-', linewidth=0.5)
            ax.grid(False, axis='x')
            
            # Spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('lightgray')
            ax.spines['bottom'].set_color('lightgray')
            
            # Legend
            ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                     ncol=2, frameon=False, fontsize=10)
            
            plt.tight_layout()
            plt.subplots_adjust(bottom=0.15)
            
            # Save
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            output_path = plots_folder / f'{metric_type}_monthly_forecast.png'
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            
            return True
            
        except Exception as e:
            print(f"Error in monthly {metric_type} for {site_name}: {str(e)}")
            return False
    
    def plot_monthly_forecast_from_stats(self, site_name, metric_type, stats_file):
        """Create monthly forecast plot using stats file (fallback)"""
        try:
            # Read the stats
            df = pd.read_csv(stats_file)
            
            # Create the plot (similar to plot_monthly_forecast but with stats data)
            fig, ax = plt.subplots(figsize=(12, 7))
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')
            
            # X positions
            x = range(len(df))
            
            # Check if this is price
            if metric_type == 'price':
                ax.plot(x, df['mean'], 
                       color=self.colors['price'],
                       linewidth=3,
                       label='Real-Time Price',
                       zorder=5,
                       marker='o',
                       markersize=6)
                
                # Try to load day-ahead price data from Price_da folder
                project_folder = self.portfolio_path / site_name
                price_da_folder = project_folder / 'Price_da'
                if price_da_folder.exists():
                    # Look for monthly stats in Price_da
                    dt_patterns = [
                        '*_monthly_stats.csv',
                        '*monthly_stats.csv',
                        '*_price_da_monthly_stats.csv'
                    ]
                    
                    dt_file = None
                    for pattern in dt_patterns:
                        files = list(price_da_folder.glob(pattern))
                        if files:
                            dt_file = files[0]
                            break
                    
                    if dt_file:
                        try:
                            df_dt = pd.read_csv(dt_file)
                            if 'mean' in df_dt.columns:
                                ax.plot(x, df_dt['mean'], 
                                       color=self.colors['price_da'],
                                       linewidth=3,
                                       label='Day-Ahead Price',
                                       zorder=5,
                                       marker='s',
                                       markersize=6,
                                       linestyle='--')
                        except Exception as e:
                            print(f"      Note: Could not load day-ahead price stats: {e}")
            else:
                # Plot with confidence band if p5/p95 available
                if 'p5' in df.columns and 'p95' in df.columns:
                    ax.fill_between(x, df['p5'], df['p95'],
                                  alpha=0.3,
                                  color=self.colors[metric_type],
                                  label='P5-P95 Confidence Band',
                                  edgecolor='none')
                
                ax.plot(x, df['mean'], 
                       color=self.colors[metric_type],
                       linewidth=3,
                       label='Mean',
                       zorder=5,
                       marker='o',
                       markersize=6)
            
            # Title
            clean_name = self.clean_site_name(site_name)
            title = f'Monthly {metric_type.capitalize()} Forecast - {clean_name}'
            ax.set_title(title, fontsize=14, fontweight='normal', pad=15)
            
            # X-axis
            ax.set_xticks(x)
            if 'month_name' in df.columns:
                ax.set_xticklabels(df['month_name'], rotation=45, ha='right')
            else:
                ax.set_xticklabels([self.month_names[i] for i in range(12)], rotation=45, ha='right')
            ax.set_xlabel('')
            
            # Y-axis
            self.format_y_axis(ax, metric_type, 'monthly')
            
            # Grid
            ax.grid(True, axis='y', alpha=0.3, linestyle='-', linewidth=0.5)
            ax.grid(False, axis='x')
            
            # Spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('lightgray')
            ax.spines['bottom'].set_color('lightgray')
            
            # Legend
            ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                     ncol=2, frameon=False, fontsize=10)
            
            plt.tight_layout()
            plt.subplots_adjust(bottom=0.15)
            
            # Save
            project_folder = self.portfolio_path / site_name
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            output_path = plots_folder / f'{metric_type}_monthly_forecast.png'
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            
            return True
            
        except Exception as e:
            print(f"Error in monthly {metric_type} from stats for {site_name}: {str(e)}")
            return False
    
    def plot_daily_forecast(self, site_name, metric_type):
        """Create daily forecast plot with 7-day rolling average"""
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
            
        # Find daily timeseries or stats file
        timeseries_file = self.find_timeseries_file(project_folder, metric_type, 'daily')
        stats_file = self.find_stats_file(project_folder, metric_type, 'daily')
        
        if not timeseries_file and not stats_file:
            return False
        
        try:
            # Prefer timeseries file, fall back to stats
            if timeseries_file:
                df = pd.read_csv(timeseries_file)
                
                # Get year columns
                year_cols = [col for col in df.columns if str(col).isdigit()]
                
                if year_cols:
                    # Calculate statistics
                    df['mean'] = df[year_cols].mean(axis=1)
                    df['p25'] = df[year_cols].quantile(0.25, axis=1)
                    df['p75'] = df[year_cols].quantile(0.75, axis=1)
            else:
                df = pd.read_csv(stats_file)
            
            # Apply 7-day rolling average
            df['mean_smooth'] = df['mean'].rolling(window=7, center=True, min_periods=4).mean()
            if 'p25' in df.columns and 'p75' in df.columns:
                df['p25_smooth'] = df['p25'].rolling(window=7, center=True, min_periods=4).mean()
                df['p75_smooth'] = df['p75'].rolling(window=7, center=True, min_periods=4).mean()
            
            # Drop NaN values
            df_smooth = df.dropna(subset=['mean_smooth'])
            
            # Create the plot
            fig, ax = plt.subplots(figsize=(16, 8))
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')
            
            # X positions
            x = range(len(df_smooth))
            
            # Check if this is price
            if metric_type == 'price':
                ax.plot(x, df_smooth['mean_smooth'], 
                       color=self.colors['price'],
                       linewidth=2.5,
                       label='Real-Time Price (7-day avg)',
                       zorder=5)
                
                # Try to load day-ahead price data from Price_da folder
                price_da_folder = project_folder / 'Price_da'
                if price_da_folder.exists():
                    # Look for daily timeseries in Price_da
                    dt_patterns = [
                        '*_daily_timeseries.csv',
                        '*daily_timeseries.csv',
                        '*_price_da_daily_timeseries.csv'
                    ]
                    
                    dt_file = None
                    for pattern in dt_patterns:
                        files = list(price_da_folder.glob(pattern))
                        if files:
                            dt_file = files[0]
                            break
                    
                    if dt_file:
                        try:
                            df_dt = pd.read_csv(dt_file)
                            year_cols_dt = [col for col in df_dt.columns if str(col).isdigit()]
                            
                            if year_cols_dt:
                                df_dt['mean'] = df_dt[year_cols_dt].mean(axis=1)
                                df_dt['mean_smooth'] = df_dt['mean'].rolling(window=7, center=True, min_periods=4).mean()
                                df_dt_smooth = df_dt.dropna(subset=['mean_smooth'])
                                
                                # Align the data if necessary
                                if len(df_dt_smooth) == len(df_smooth):
                                    ax.plot(x, df_dt_smooth['mean_smooth'], 
                                           color=self.colors['price_da'],
                                           linewidth=2.5,
                                           label='Day-Ahead Price (7-day avg)',
                                           zorder=5,
                                           linestyle='--')
                        except Exception as e:
                            print(f"      Note: Could not load day-ahead price data: {e}")
                
            else:
                # Plot with confidence band if available
                if 'p25_smooth' in df_smooth.columns and 'p75_smooth' in df_smooth.columns:
                    ax.fill_between(x, 
                                  df_smooth['p25_smooth'], 
                                  df_smooth['p75_smooth'],
                                  alpha=0.3,
                                  color=self.colors[metric_type],
                                  label='P25-P75 Confidence Band (7-day avg)',
                                  edgecolor='none')
                
                ax.plot(x, df_smooth['mean_smooth'], 
                       color=self.colors[metric_type],
                       linewidth=2.5,
                       label='Mean (7-day avg)',
                       zorder=5)
            
            # Title
            clean_name = self.clean_site_name(site_name)
            title = f'Daily {metric_type.capitalize()} Forecast (7-day Rolling Average) - {clean_name}'
            ax.set_title(title, fontsize=14, fontweight='normal', pad=15)
            
            # X-axis - show every 30th day
            tick_positions = list(range(0, len(df_smooth), 30))
            if 'date_label' in df_smooth.columns:
                tick_labels = [df_smooth.iloc[i]['date_label'] for i in tick_positions]
            else:
                # Create month-day labels
                tick_labels = []
                for i in tick_positions:
                    month = int(df_smooth.iloc[i]['month'])
                    day = int(df_smooth.iloc[i]['day'])
                    tick_labels.append(f"{self.month_names[month-1][:3]} {day}")
            
            ax.set_xticks(tick_positions)
            ax.set_xticklabels(tick_labels, rotation=45, ha='right')
            ax.set_xlabel('')
            
            # Y-axis
            self.format_y_axis(ax, metric_type, 'daily')
            
            # Grid
            ax.grid(True, axis='y', alpha=0.3, linestyle='-', linewidth=0.5)
            ax.grid(True, axis='x', alpha=0.2, linestyle=':', linewidth=0.5)
            
            # Spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('lightgray')
            ax.spines['bottom'].set_color('lightgray')
            
            # Legend
            ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12),
                     ncol=2, frameon=False, fontsize=10)
            
            plt.tight_layout()
            plt.subplots_adjust(bottom=0.12)
            
            # Save
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            output_path = plots_folder / f'{metric_type}_daily_forecast.png'
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            
            return True
            
        except Exception as e:
            print(f"Error in daily {metric_type} for {site_name}: {str(e)}")
            return False
    
    def plot_hourly_forecast(self, site_name, metric_type):
        """Create hourly average profile"""
        # Skip hourly profile for revenue
        if metric_type == 'revenue':
            return False
        
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
            
        # Find hourly timeseries or stats file
        timeseries_file = self.find_timeseries_file(project_folder, metric_type, 'hourly')
        stats_file = self.find_stats_file(project_folder, metric_type, 'hourly')
        
        if not timeseries_file and not stats_file:
            return False
        
        try:
            # Prefer timeseries file
            if timeseries_file:
                df = pd.read_csv(timeseries_file)
                
                # Get year columns
                year_cols = [col for col in df.columns if str(col).isdigit()]
                
                if year_cols:
                    # Calculate statistics
                    df['mean'] = df[year_cols].mean(axis=1)
                    df['p5'] = df[year_cols].quantile(0.05, axis=1)
                    df['p95'] = df[year_cols].quantile(0.95, axis=1)
                
                # Extract hour if needed
                if 'hour' not in df.columns:
                    if 'datetime' in df.columns or 'timestamp' in df.columns:
                        time_col = 'datetime' if 'datetime' in df.columns else 'timestamp'
                        df[time_col] = pd.to_datetime(df[time_col])
                        df['hour'] = df[time_col].dt.hour
                    else:
                        df['hour'] = df.index % 24
                
                # Group by hour
                hourly_profile = df.groupby('hour').agg({
                    'mean': 'mean',
                    'p5': 'mean',
                    'p95': 'mean'
                }).reset_index()
            else:
                df = pd.read_csv(stats_file)
                if 'hour' in df.columns:
                    hourly_profile = df.groupby('hour').agg({
                        'mean': 'mean',
                        'p5': 'mean',
                        'p95': 'mean'
                    }).reset_index()
                else:
                    return False
            
            # Create the plot
            fig, ax = plt.subplots(figsize=(10, 6))
            ax.set_facecolor('white')
            fig.patch.set_facecolor('white')
            
            # Check if this is price
            if metric_type == 'price':
                # Plot with confidence band for real-time price
                if 'p5' in hourly_profile.columns and 'p95' in hourly_profile.columns:
                    ax.fill_between(hourly_profile['hour'], 
                                  hourly_profile['p5'], 
                                  hourly_profile['p95'],
                                  alpha=0.3,
                                  color=self.colors['price'],
                                  label='P5-P95 RT Price',
                                  edgecolor='none')
                
                ax.plot(hourly_profile['hour'], hourly_profile['mean'], 
                       color=self.colors['price'],
                       linewidth=3,
                       marker='o',
                       markersize=6,
                       label='Real-Time Price Mean',
                       zorder=5)
                
                # Try to load day-ahead price data from Price_da folder
                price_da_folder = project_folder / 'Price_da'
                if price_da_folder.exists():
                    # Look for hourly timeseries in Price_da
                    dt_patterns = [
                        '*_hourly_timeseries.csv',
                        '*hourly_timeseries.csv',
                        '*_hourly_stats.csv',
                        '*hourly_stats.csv'
                    ]
                    
                    dt_file = None
                    for pattern in dt_patterns:
                        files = list(price_da_folder.glob(pattern))
                        if files:
                            dt_file = files[0]
                            break
                    
                    if dt_file:
                        try:
                            df_dt = pd.read_csv(dt_file)
                            
                            # Process based on file type
                            if 'timeseries' in dt_file.name:
                                year_cols_dt = [col for col in df_dt.columns if str(col).isdigit()]
                                if year_cols_dt:
                                    df_dt['mean'] = df_dt[year_cols_dt].mean(axis=1)
                                    
                                    if 'hour' not in df_dt.columns:
                                        df_dt['hour'] = df_dt.index % 24
                                    
                                    hourly_profile_dt = df_dt.groupby('hour')['mean'].mean().reset_index()
                            else:
                                # Stats file
                                if 'hour' in df_dt.columns:
                                    hourly_profile_dt = df_dt.groupby('hour')['mean'].mean().reset_index()
                                else:
                                    hourly_profile_dt = None
                            
                            if hourly_profile_dt is not None:
                                ax.plot(hourly_profile_dt['hour'], hourly_profile_dt['mean'], 
                                       color=self.colors['price_da'],
                                       linewidth=3,
                                       marker='s',
                                       markersize=6,
                                       label='Day-Ahead Price Mean',
                                       zorder=5,
                                       linestyle='--')
                        except Exception as e:
                            print(f"      Note: Could not load day-ahead hourly price data: {e}")
            else:
                # For generation, plot with confidence band
                if 'p5' in hourly_profile.columns and 'p95' in hourly_profile.columns:
                    ax.fill_between(hourly_profile['hour'], 
                                  hourly_profile['p5'], 
                                  hourly_profile['p95'],
                                  alpha=0.3,
                                  color=self.colors[metric_type],
                                  label='P5-P95 Confidence Band',
                                  edgecolor='none')
                
                ax.plot(hourly_profile['hour'], hourly_profile['mean'], 
                       color=self.colors[metric_type],
                       linewidth=3,
                       marker='o',
                       markersize=6,
                       label='Mean',
                       zorder=5)
            
            # Title
            clean_name = self.clean_site_name(site_name)
            title = f'Average Hourly {metric_type.capitalize()} Profile - {clean_name}'
            ax.set_title(title, fontsize=14, fontweight='normal', pad=15)
            
            # X-axis
            ax.set_xlabel('Hour of Day', fontsize=11)
            ax.set_xticks(range(0, 24, 2))
            ax.set_xlim(-0.5, 23.5)
            
            # Y-axis
            self.format_y_axis(ax, metric_type, 'hourly')
            
            # Grid
            ax.grid(True, axis='y', alpha=0.3, linestyle='-', linewidth=0.5)
            ax.grid(True, axis='x', alpha=0.2, linestyle=':', linewidth=0.5)
            
            # Spines
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['left'].set_color('lightgray')
            ax.spines['bottom'].set_color('lightgray')
            
            # Legend
            ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
                     ncol=2, frameon=False, fontsize=10)
            
            plt.tight_layout()
            plt.subplots_adjust(bottom=0.15)
            
            # Save
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            output_path = plots_folder / f'{metric_type}_hourly_profile.png'
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            
            return True
            
        except Exception as e:
            print(f"Error in hourly {metric_type} for {site_name}: {str(e)}")
            return False
    
    def create_combined_forecast(self, site_name):
        """Create a combined plot showing all three metrics for a site"""
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
        
        # Check which metrics are available
        metrics_available = []
        for metric in ['generation', 'price', 'revenue']:
            metric_folder = project_folder / metric.capitalize()
            if metric_folder.exists():
                # Check if any monthly files exist
                monthly_files = list(metric_folder.glob('*monthly*.csv'))
                if monthly_files:
                    metrics_available.append(metric)
        
        if len(metrics_available) < 2:  # Need at least 2 metrics
            return False
        
        try:
            # Create figure with subplots
            fig, axes = plt.subplots(len(metrics_available), 1, 
                                   figsize=(12, 4*len(metrics_available)), 
                                   sharex=True)
            fig.patch.set_facecolor('white')
            
            if len(metrics_available) == 1:
                axes = [axes]
            
            for idx, metric in enumerate(metrics_available):
                ax = axes[idx]
                ax.set_facecolor('white')
                
                # Find monthly data
                timeseries_file = self.find_timeseries_file(project_folder, metric, 'monthly')
                
                if not timeseries_file:
                    continue
                
                # Read data
                df = pd.read_csv(timeseries_file)
                
                # Get year columns
                year_cols = [col for col in df.columns if str(col).isdigit()]
                
                if year_cols:
                    # Calculate statistics
                    df['mean'] = df[year_cols].mean(axis=1)
                    df['p5'] = df[year_cols].quantile(0.05, axis=1)
                    df['p95'] = df[year_cols].quantile(0.95, axis=1)
                
                # X positions
                x = range(len(df))
                
                # Plot based on metric type
                if metric == 'price':
                    ax.plot(x, df['mean'], 
                           color=self.colors['price'],
                           linewidth=3,
                           label='Real-Time Price',
                           marker='o',
                           markersize=5)
                    
                    # Try to add day-ahead price
                    price_da_folder = project_folder / 'Price_da'
                    if price_da_folder.exists():
                        dt_patterns = [
                            '*_monthly_timeseries.csv',
                            '*monthly_timeseries.csv',
                            '*_price_da_monthly_timeseries.csv'
                        ]
                        
                        dt_file = None
                        for pattern in dt_patterns:
                            files = list(price_da_folder.glob(pattern))
                            if files:
                                dt_file = files[0]
                                break
                        
                        if dt_file:
                            try:
                                df_dt = pd.read_csv(dt_file)
                                year_cols_dt = [col for col in df_dt.columns if str(col).isdigit()]
                                
                                if year_cols_dt:
                                    df_dt['mean'] = df_dt[year_cols_dt].mean(axis=1)
                                    ax.plot(x, df_dt['mean'], 
                                           color=self.colors['price_da'],
                                           linewidth=3,
                                           label='Day-Ahead Price',
                                           marker='s',
                                           markersize=5,
                                           linestyle='--')
                            except:
                                pass
                else:
                    # Plot with confidence band
                    if 'p5' in df.columns and 'p95' in df.columns:
                        ax.fill_between(x, df['p5'], df['p95'],
                                      alpha=0.3,
                                      color=self.colors[metric],
                                      label='P5-P95')
                    
                    ax.plot(x, df['mean'], 
                           color=self.colors[metric],
                           linewidth=3,
                           label='Mean',
                           marker='o',
                           markersize=5)
                
                # Subplot title
                subplot_titles = {
                    'generation': 'Monthly Generation (MWh)',
                    'price': 'Monthly Price ($/MWh)',
                    'revenue': 'Monthly Revenue ($)'
                }
                ax.set_title(subplot_titles[metric], fontsize=12, pad=10)
                
                # Y-axis
                self.format_y_axis(ax, metric, 'monthly')
                
                # Grid
                ax.grid(True, axis='y', alpha=0.3, linestyle='-', linewidth=0.5)
                ax.grid(False, axis='x')
                
                # Spines
                ax.spines['top'].set_visible(False)
                ax.spines['right'].set_visible(False)
                ax.spines['left'].set_color('lightgray')
                ax.spines['bottom'].set_color('lightgray')
                
                # Legend
                ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.25),
                         ncol=2, frameon=False, fontsize=9)
                
                # X-axis labels only on bottom plot
                if idx == len(metrics_available) - 1:
                    ax.set_xticks(x)
                    if 'month_name' in df.columns:
                        ax.set_xticklabels(df['month_name'], rotation=45, ha='right')
                    else:
                        ax.set_xticklabels([self.month_names[i] for i in range(12)], rotation=45, ha='right')
            
            # Main title
            clean_name = self.clean_site_name(site_name)
            fig.suptitle(f'Combined Monthly Forecasts - {clean_name}', 
                        fontsize=16, fontweight='normal', y=0.995)
            
            plt.tight_layout()
            plt.subplots_adjust(hspace=0.5, top=0.96)
            
            # Save
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            output_path = plots_folder / 'combined_monthly_forecast.png'
            plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            
            return True
            
        except Exception as e:
            print(f"Error in combined forecast for {site_name}: {str(e)}")
            return False
    
    def run_all_visualizations(self):
        """Process all sites and create all visualization types"""
        print("\n🌟 STATISTICS-BASED VISUALIZATION SYSTEM")
        print("="*60)
        print(f"📁 Portfolio folder: {self.portfolio_path}")
        print("📁 Plots will be saved in each site's 'plots' subfolder")
        print("="*60)
        
        # Check if portfolio path exists
        if not self.portfolio_path.exists():
            print(f"❌ Error: Portfolio path '{self.portfolio_path}' not found!")
            print("\n   Please check:")
            print("   1. You are in the correct directory")
            print("   2. The 'Renewable Portfolio LLC' folder exists")
            return
        
        # Get all sites
        sites = self.get_all_sites()
        
        if not sites:
            print("❌ No project folders found!")
            print("\n   Expected folder structure:")
            print("   Renewable Portfolio LLC/")
            print("     ├── Project_Name/")
            print("     │   ├── Generation/")
            print("     │   ├── Price/")
            print("     │   ├── Price_da/  (optional)")
            print("     │   └── Revenue/")
            return
        
        print(f"\n📊 Found {len(sites)} sites to process:")
        for site in sites:
            print(f"   - {self.clean_site_name(site)}")
        
        # Process each site
        for site_idx, site_name in enumerate(sites, 1):
            clean_name = self.clean_site_name(site_name)
            print(f"\n[{site_idx}/{len(sites)}] Processing: {clean_name}")
            print("-"*40)
            
            # Create plots folder inside the site folder
            project_folder = self.portfolio_path / site_name
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            
            # Track what was created
            created_plots = []
            
            # Process each metric and temporal resolution
            for metric in ['generation', 'price', 'revenue']:
                # Monthly forecasts
                if self.plot_monthly_forecast(site_name, metric):
                    created_plots.append(f"{metric}_monthly_forecast.png")
                    print(f"   ✅ {metric.capitalize()} monthly forecast")
                    if metric == 'price' and (project_folder / 'Price_da').exists():
                        print(f"      (includes day-ahead price data)")
                
                # Daily forecasts (with 7-day rolling average)
                if self.plot_daily_forecast(site_name, metric):
                    created_plots.append(f"{metric}_daily_forecast.png")
                    print(f"   ✅ {metric.capitalize()} daily forecast (7-day avg)")
                    if metric == 'price' and (project_folder / 'Price_da').exists():
                        print(f"      (includes day-ahead price data)")
                
                # Hourly profiles
                if self.plot_hourly_forecast(site_name, metric):
                    created_plots.append(f"{metric}_hourly_profile.png")
                    print(f"   ✅ {metric.capitalize()} hourly profile")
                    if metric == 'price' and (project_folder / 'Price_da').exists():
                        print(f"      (includes day-ahead price data)")
                elif metric == 'revenue':
                    print(f"   ⏭️  {metric.capitalize()} hourly profile (skipped)")
                
                # Monthly duration curves for PRICE ONLY
                if metric == 'price':
                    if self.plot_monthly_duration_curves(site_name, metric):
                        created_plots.append(f"duration_curves/ (12 monthly files)")
                        print(f"   ✅ {metric.capitalize()} monthly duration curves (12 plots)")
                
                # Monthly distribution curves for GENERATION and REVENUE
                if metric in ['generation', 'revenue']:
                    if self.plot_monthly_distribution_curves(site_name, metric):
                        created_plots.append(f"distribution_curves/ (12 monthly files)")
                        print(f"   ✅ {metric.capitalize()} monthly distribution curves (12 plots)")
            
            # Combined forecast
            if self.create_combined_forecast(site_name):
                created_plots.append("combined_monthly_forecast.png")
                print(f"   ✅ Combined monthly forecast")
            
            print(f"\n   📁 Created {len(created_plots)} visualizations in: {site_name}/plots/")
        
        print("\n" + "="*60)
        print("✨ VISUALIZATION COMPLETE!")
        print(f"📁 All plots saved in each site's 'plots' subfolder")
        print("\nFolder structure created:")
        for site in sites[:3]:  # Show first 3 as example
            print(f"   📁 Renewable Portfolio LLC/{site}/")
            print(f"      📁 Generation/")
            print(f"      📁 Price/")
            print(f"      📁 Price_da/  (if available)")
            print(f"      📁 Revenue/")
            print(f"      📁 plots/  ← All visualizations here")
            print(f"         📊 generation_monthly_forecast.png")
            print(f"         📊 price_monthly_forecast.png (with day-ahead if available)")
            print(f"         📊 revenue_monthly_forecast.png")
            print(f"         📊 combined_monthly_forecast.png")
            print(f"         📁 duration_curves/ (price only)")
            print(f"         📁 distribution_curves/ (generation & revenue)")
        if len(sites) > 3:
            print(f"   ... and {len(sites)-3} more sites")
        print("\n📌 Note: Price charts include both real-time and day-ahead data when Price_da folder exists")
        print("="*60)

# Run the visualization
if __name__ == "__main__":
    print("🚀 Starting Renewable Portfolio Visualization System\n")
    
    # Create visualizer instance
    visualizer = StatsBasedVisualization()
    
    # Run visualizations
    visualizer.run_all_visualizations()

🚀 Starting Renewable Portfolio Visualization System

✅ Found portfolio at: Renewable Portfolio LLC

🌟 STATISTICS-BASED VISUALIZATION SYSTEM
📁 Portfolio folder: Renewable Portfolio LLC
📁 Plots will be saved in each site's 'plots' subfolder

📊 Found 7 sites to process:
   - Blue Wing Solar Energy Generator
   - High Lonesome Wind
   - Midway Solar Farm Iii
   - Misae Solar
   - Mount Signal Solar Farm Ii
   - Re Mustang
   - Stanton Wind Energy

[1/7] Processing: Blue Wing Solar Energy Generator
----------------------------------------
   ✅ Generation monthly forecast
   ✅ Generation daily forecast (7-day avg)
   ✅ Generation hourly profile
   ✅ Generation monthly distribution curves (12 plots)
   ✅ Price monthly forecast
      (includes day-ahead price data)
   ✅ Price daily forecast (7-day avg)
      (includes day-ahead price data)
   ✅ Price hourly profile
      (includes day-ahead price data)
   ✅ Price monthly duration curves (12 plots)
   ✅ Revenue monthly forecast
   ✅ Revenue dai

# scatter plots 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import LinearSegmentedColormap
from pathlib import Path
import numpy as np
import warnings
warnings.filterwarnings('ignore')

class WeatherGenerationCorrelation:
    """
    Create GHI vs Generation correlation plots for solar sites
    """
    
    def __init__(self):
        # Local paths - assuming this runs from the same directory as plots.ipynb
        self.base_path = Path('.')  # Current directory
        self.portfolio_path = self.base_path / 'Renewable Portfolio LLC'
        self.weather_data_path = self.base_path / 'weather_data'
        
        # Check if paths exist
        if self.portfolio_path.exists():
            print(f"✅ Found portfolio at: {self.portfolio_path}")
        else:
            print(f"⚠️  Portfolio path not found at: {self.portfolio_path}")
            print("   Please verify you're running this from the correct directory")
            
        # Check weather data path
        if self.weather_data_path.exists():
            print(f"✅ Found weather data at: {self.weather_data_path}")
        else:
            print(f"⚠️  Weather data path not found at: {self.weather_data_path}")
    
    def get_project_folders(self):
        """Get all project folders (excluding notebooks and other files)"""
        project_folders = []
        if not self.portfolio_path.exists():
            return project_folders
            
        for item in self.portfolio_path.iterdir():
            if item.is_dir():
                # Check if it has the expected subfolders
                if ((item / 'Generation').exists() or (item / 'Price').exists() or 
                    (item / 'Price_da').exists() or (item / 'Revenue').exists()):
                    project_folders.append(item)
        return project_folders
    
    def clean_site_name(self, site_name):
        """Clean up site name for display"""
        # Remove LLC suffixes and underscores for cleaner display
        clean_name = site_name.replace('_LLC', '').replace('_Power', '')
        clean_name = clean_name.replace('_', ' ').title()
        return clean_name
    
    def get_all_sites(self):
        """Get all unique site names from the project folders"""
        return [folder.name for folder in self.get_project_folders()]
    
    def plot_weather_generation_correlation(self, site_name):
        """Create GHI vs Generation correlation plots for a single solar site"""
        # Skip wind sites (they won't have meaningful solar correlations)
        if 'wind' in site_name.lower():
            print(f"   ⏭️  Skipping {site_name} (wind site)")
            return False
            
        # Get project folder
        project_folder = self.portfolio_path / site_name
        if not project_folder.exists():
            return False
        
        # UPDATED: Look for weather file in weather_data folder
        weather_file_path = self.weather_data_path / f'{site_name}_weather_hourly.csv'
        if not weather_file_path.exists():
            print(f"   ❌ No weather file found at {weather_file_path}")
            return False
            
        # Find generation hourly timeseries
        gen_folder = project_folder / 'Generation'
        if not gen_folder.exists():
            print(f"   ❌ No Generation folder found for {site_name}")
            return False
            
        gen_files = list(gen_folder.glob('*_hourly_timeseries.csv'))
        if not gen_files:
            print(f"   ❌ No hourly generation timeseries found for {site_name}")
            return False
            
        try:
            print(f"   🔄 Processing weather-generation correlation...")
            
            # Read weather data from the weather_data folder
            weather_df = pd.read_csv(weather_file_path)
            print(f"      📊 Loaded weather data: {len(weather_df)} rows")
            
            # Adjust weather hour from HE to HB (subtract 1)
            weather_df['HOUR'] = weather_df['HOUR'] - 1
            
            # Handle hour 0 case (hour 0 in HE becomes hour 23 of previous day)
            # For now, we'll drop these as they would need previous day's data
            weather_df = weather_df[weather_df['HOUR'] >= 0].copy()
            
            # Convert DATE to datetime
            weather_df['date'] = pd.to_datetime(weather_df['DATE'])
            weather_df['year'] = weather_df['date'].dt.year
            weather_df['month'] = weather_df['date'].dt.month
            weather_df['day'] = weather_df['date'].dt.day
            weather_df['hour'] = weather_df['HOUR']
            
            # Read generation data
            gen_df = pd.read_csv(gen_files[0])
            print(f"      📊 Loaded generation data: {len(gen_df)} rows")
            
            # Get year columns (these contain the generation values)
            year_cols = [col for col in gen_df.columns if str(col).isdigit()]
            
            if not year_cols:
                print(f"   ❌ No year columns found in generation data")
                return False
                
            print(f"      📅 Found {len(year_cols)} years of generation data")
            
            # Process each year
            all_data = []
            
            for year_col in year_cols:
                # Skip if not numeric year
                try:
                    year_val = int(year_col)
                except:
                    continue
                    
                # Create a copy of generation data for this year
                gen_year = gen_df[['month', 'day', 'hour', year_col]].copy()
                gen_year['year'] = year_val
                gen_year['generation_mw'] = pd.to_numeric(gen_year[year_col], errors='coerce')
                
                # Filter weather data for this year
                weather_year = weather_df[weather_df['year'] == year_val].copy()
                
                if len(weather_year) == 0:
                    continue
                
                # Merge on year, month, day, hour
                merged = pd.merge(
                    gen_year[['year', 'month', 'day', 'hour', 'generation_mw']],
                    weather_year[['year', 'month', 'day', 'hour', 'shortwave_radiation', 'temperature_2m']],
                    on=['year', 'month', 'day', 'hour'],
                    how='inner'
                )
                
                # Rename columns for clarity
                merged['ghi'] = merged['shortwave_radiation']
                merged['temperature'] = merged['temperature_2m']
                
                # Remove invalid data
                merged = merged[
                    (merged['generation_mw'] >= 0) & 
                    (merged['ghi'] >= 0) &
                    (~merged['generation_mw'].isna()) &
                    (~merged['ghi'].isna()) &
                    (~merged['temperature'].isna())
                ]
                
                if len(merged) > 0:
                    all_data.append(merged)
                    print(f"      ✓ Year {year_val}: {len(merged)} valid data points")
            
            if not all_data:
                print(f"   ❌ No valid merged data found")
                return False
                
            # Combine all years
            combined_df = pd.concat(all_data, ignore_index=True)
            print(f"      📊 Total combined data points: {len(combined_df)}")
            
            # Filter out nighttime hours (where generation is essentially zero)
            # Keep only hours where there's meaningful generation (> 0.1 MW)
            combined_df = combined_df[combined_df['generation_mw'] > 0.1].copy()
            print(f"      📊 Data points after filtering low generation: {len(combined_df)}")
            
            if len(combined_df) == 0:
                print(f"   ❌ No data points with generation > 0.1 MW")
                return False
            
            # Get the actual hour range in the data AFTER filtering
            hour_min = int(combined_df['hour'].min())
            hour_max = int(combined_df['hour'].max())
            print(f"      📊 Hour range in data: {hour_min} to {hour_max}")
            
            # Create plots folder if it doesn't exist (it should already exist)
            plots_folder = project_folder / 'plots'
            plots_folder.mkdir(exist_ok=True)
            
            # Get clean site name
            clean_name = self.clean_site_name(site_name)
            
            # Create two plots
            created_plots = []
            
            # Plot 1: GHI vs Generation colored by Hour
            print(f"      🎨 Creating GHI vs Generation (by hour) plot...")
            fig1, ax1 = plt.subplots(figsize=(10, 8))
            ax1.set_facecolor('white')
            fig1.patch.set_facecolor('white')
            
            # Create custom colormap that matches reference
            # The reference shows distinct color bands throughout the day
            # Using a more vibrant color scheme with clear transitions
            
            # Full day color progression (will be subset based on actual hours)
            full_day_colors = [
                '#1a0033',  # 0-1: Very dark purple (midnight)
                '#1a0033',  # 1-2
                '#1a0033',  # 2-3
                '#1a0033',  # 3-4
                '#2d1b69',  # 4-5: Dark purple (pre-dawn)
                '#483d8b',  # 5-6: Dark slate blue
                '#0047ab',  # 6-7: Cobalt blue (dawn)
                '#0080ff',  # 7-8: Light blue
                '#00bfff',  # 8-9: Deep sky blue
                '#40e0d0',  # 9-10: Turquoise
                '#ff0000',  # 10-11: Red (late morning)
                '#ff4500',  # 11-12: Orange red (noon)
                '#ff6347',  # 12-13: Tomato
                '#ff8c00',  # 13-14: Dark orange
                '#ffa500',  # 14-15: Orange
                '#ffd700',  # 15-16: Gold
                '#ffff00',  # 16-17: Yellow
                '#adff2f',  # 17-18: Green yellow
                '#9acd32',  # 18-19: Yellow green
                '#32cd32',  # 19-20: Lime green
                '#228b22',  # 20-21: Forest green
                '#006400',  # 21-22: Dark green
                '#1a0033',  # 22-23: Back to dark purple
                '#1a0033'   # 23-24
            ]
            
            # Extract only the colors for hours present in the data
            colors = []
            for h in range(hour_min, hour_max + 1):
                if h < len(full_day_colors):
                    colors.append(full_day_colors[h])
                else:
                    colors.append(full_day_colors[-1])
            
            # Create colormap
            cmap_custom = LinearSegmentedColormap.from_list('custom_hours', colors, N=len(colors))
            
            # Scatter plot with hour coloring
            scatter1 = ax1.scatter(
                combined_df['ghi'],
                combined_df['generation_mw'],
                c=combined_df['hour'],
                cmap=cmap_custom,
                s=2,  # Slightly larger points for better visibility
                alpha=0.7,  # Slightly higher alpha for more vibrant colors
                edgecolors='none',
                vmin=hour_min,
                vmax=hour_max,
                rasterized=True  # Better performance for many points
            )
            
            # Colorbar with adjusted range
            cbar1 = plt.colorbar(scatter1, ax=ax1)
            cbar1.set_label('Hour of Day', rotation=270, labelpad=20, fontsize=12)
            
            # Set colorbar ticks to show only hours with data
            if hour_max - hour_min > 12:
                # If wide range, show every 2 hours
                hour_ticks = list(range(hour_min, hour_max + 1, 2))
            else:
                # If narrow range, show every hour
                hour_ticks = list(range(hour_min, hour_max + 1))
            cbar1.set_ticks(hour_ticks)
            cbar1.set_ticklabels(hour_ticks)
            
            # Labels and title
            ax1.set_xlabel('Global Horizontal Irradiance (W/m²)', fontsize=12)
            ax1.set_ylabel('Modeled Solar Generation (MW)', fontsize=12)
            ax1.set_title(f'{clean_name} (All Years)\nModeled Generation vs. Global Horizontal Irradiance', 
                         fontsize=14, fontweight='normal', pad=15)
            
            # Add metadata box
            data_points = len(combined_df)
            max_gen = combined_df['generation_mw'].max()
            max_ghi = combined_df['ghi'].max()
            hour_range = f"{hour_min}-{hour_max}"
            
            textstr1 = f'Data Points: {data_points:,}\nMax Generation: {max_gen:.2f} MW\nMax GHI: {max_ghi:.1f} W/m²\nHour Range: {hour_range}'
            props = dict(boxstyle='round', facecolor='white', alpha=0.8, edgecolor='gray')
            ax1.text(0.02, 0.98, textstr1, transform=ax1.transAxes, fontsize=10,
                    verticalalignment='top', bbox=props)
            
            # Grid
            ax1.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
            
            # Set limits
            ax1.set_xlim(0, combined_df['ghi'].max() * 1.05)
            ax1.set_ylim(0, combined_df['generation_mw'].max() * 1.05)
            
            plt.tight_layout()
            
            # Save plot 1
            plot1_path = plots_folder / f'ghi_vs_generation_by_hour.png'
            plt.savefig(plot1_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            created_plots.append('ghi_vs_generation_by_hour.png')
            
            # Plot 2: GHI vs Generation colored by Temperature
            print(f"      🎨 Creating GHI vs Generation (by temperature) plot...")
            fig2, ax2 = plt.subplots(figsize=(10, 8))
            ax2.set_facecolor('white')
            fig2.patch.set_facecolor('white')
            
            # Scatter plot with temperature coloring
            scatter2 = ax2.scatter(
                combined_df['ghi'],
                combined_df['generation_mw'],
                c=combined_df['temperature'],
                cmap='coolwarm',
                s=2,  # Slightly larger points for better visibility
                alpha=0.7,  # Slightly higher alpha
                edgecolors='none',
                rasterized=True  # Better performance for many points
            )
            
            # Colorbar
            cbar2 = plt.colorbar(scatter2, ax=ax2)
            cbar2.set_label('Temperature (°C)', rotation=270, labelpad=20, fontsize=12)
            
            # Labels and title
            ax2.set_xlabel('Global Horizontal Irradiance (W/m²)', fontsize=12)
            ax2.set_ylabel('Modeled Solar Generation (MW)', fontsize=12)
            ax2.set_title(f'{clean_name} (All Years)\nModeled Generation vs. GHI (Colored by Temperature)', 
                         fontsize=14, fontweight='normal', pad=15)
            
            # Add metadata box
            temp_min = combined_df['temperature'].min()
            temp_max = combined_df['temperature'].max()
            
            textstr2 = f'Data Points: {data_points:,}\nMax Generation: {max_gen:.2f} MW\nMax GHI: {max_ghi:.1f} W/m²\nTemp Range: {temp_min:.1f}°C - {temp_max:.1f}°C\nHour Range: {hour_range}'
            ax2.text(0.02, 0.98, textstr2, transform=ax2.transAxes, fontsize=10,
                    verticalalignment='top', bbox=props)
            
            # Grid
            ax2.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
            
            # Set limits
            ax2.set_xlim(0, combined_df['ghi'].max() * 1.05)
            ax2.set_ylim(0, combined_df['generation_mw'].max() * 1.05)
            
            plt.tight_layout()
            
            # Save plot 2
            plot2_path = plots_folder / f'ghi_vs_generation_by_temperature.png'
            plt.savefig(plot2_path, dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            created_plots.append('ghi_vs_generation_by_temperature.png')
            
            print(f"      ✅ Created {len(created_plots)} correlation plots")
            print(f"      📁 Saved to: {plots_folder}")
            return True
            
        except Exception as e:
            print(f"   ❌ Error: {str(e)}")
            import traceback
            traceback.print_exc()
            return False
    
    def run_all_sites(self):
        """Process all solar sites and create weather-generation correlation plots"""
        print("\n🌟 WEATHER-GENERATION CORRELATION ANALYSIS")
        print("="*60)
        print("Creating GHI vs Generation plots for solar sites")
        print("="*60)
        
        # Check if portfolio path exists
        if not self.portfolio_path.exists():
            print(f"❌ Error: Portfolio path '{self.portfolio_path}' not found!")
            return
        
        # Get all sites
        sites = self.get_all_sites()
        
        if not sites:
            print("❌ No project folders found!")
            return
        
        # Filter for solar sites
        solar_sites = [site for site in sites if 'wind' not in site.lower()]
        
        print(f"\n📊 Found {len(solar_sites)} solar sites to process:")
        for site in solar_sites:
            print(f"   - {self.clean_site_name(site)}")
        
        # Process each solar site
        successful = 0
        for site_idx, site_name in enumerate(solar_sites, 1):
            clean_name = self.clean_site_name(site_name)
            print(f"\n[{site_idx}/{len(solar_sites)}] Processing: {clean_name}")
            print("-"*40)
            
            if self.plot_weather_generation_correlation(site_name):
                successful += 1
        
        print("\n" + "="*60)
        print(f"✨ ANALYSIS COMPLETE!")
        print(f"📊 Successfully created plots for {successful}/{len(solar_sites)} solar sites")
        print(f"📁 Plots saved in each site's 'plots' subfolder:")
        print(f"   - ghi_vs_generation_by_hour.png")
        print(f"   - ghi_vs_generation_by_temperature.png")
        print("="*60)

# Run the weather-generation correlation analysis
if __name__ == "__main__":
    print("🚀 Starting Weather-Generation Correlation Analysis\n")
    
    # Create analyzer instance
    analyzer = WeatherGenerationCorrelation()
    
    # Run analysis for all sites
    analyzer.run_all_sites()

🚀 Starting Weather-Generation Correlation Analysis

✅ Found portfolio at: Renewable Portfolio LLC
✅ Found weather data at: weather_data

🌟 WEATHER-GENERATION CORRELATION ANALYSIS
Creating GHI vs Generation plots for solar sites

📊 Found 5 solar sites to process:
   - Blue Wing Solar Energy Generator
   - Midway Solar Farm Iii
   - Misae Solar
   - Mount Signal Solar Farm Ii
   - Re Mustang

[1/5] Processing: Blue Wing Solar Energy Generator
----------------------------------------
   🔄 Processing weather-generation correlation...
      📊 Loaded weather data: 482136 rows
      📊 Loaded generation data: 8784 rows
      📅 Found 14 years of generation data
      ✓ Year 2012: 7033 valid data points
      ✓ Year 2013: 8391 valid data points
      ✓ Year 2014: 8390 valid data points
      ✓ Year 2015: 8394 valid data points
      ✓ Year 2016: 8417 valid data points
      ✓ Year 2017: 8394 valid data points
      ✓ Year 2018: 8387 valid data points
      ✓ Year 2019: 8369 valid data points
   